## Housekeeping for the class. 
If you are using WSL (Windows Subsystem for Linux), you will need to reactivate your conda environment from within the _correct_ WSL distribution. You can do this by opening a terminal and running:

```bash
wsl -l -v
```
This will list all your WSL distributions and their versions. Make sure you are in the correct one where you set up your conda environment. To change into the correct distribution, you can use:

```bash
wsl -d <distribution_name>
```
Load the correct distribution and then activate your conda environment:
```bash
conda activate atomistic-ml-class
```

We will also be using `OVITO` in this class for visualization. If you have not installed it yet, you can do so by following the instructions on the [OVITO website](https://www.ovito.org/#download).


## Part 1

## Introduction to machine learning interatomic potentials and training with graph neural networks

In this part, we will learn how to train and use machine learning interatomic potentials (MLIPs) to run molecular dynamics. Specifically we will use graph neural networks (GNNs) as our frameworks for our MLIPs, focusing on two lightweight yet expressive models: NequIP and SchNet architectures. You can find the original papers for these architectures here:

- [NequIP](https://www.nature.com/articles/s41467-022-29939-5)
- [SchNet](https://pubs.aip.org/aip/jcp/article/148/24/241722/962591/SchNet-A-deep-learning-architecture-for-molecules)

Unlike descriptor-based models, GNNs learn directly from atomic graphs with position and feature information.

Both can be trained on consumer hardware (e.g., laptops), making them suitable for hands-on learning.

We will be using graph-pes to handle training and running the models. graph-pes is a Python package that provides a simple interface for training and using GNNs for interatomic potentials. You can find the documentation for graph-pes here:

- [graph-pes documentation](https://jla-gardner.github.io/graph-pes/)

Take a look at the training documentation here:

- [graph-pes training documentation](https://jla-gardner.github.io/graph-pes/cli/graph-pes-train/complete-docs.html)


In [1]:
from load_atoms import load_dataset

# load structures and split the data into training, validation and test
structures = load_dataset("../Class-1/structures_filt.xyz")

# alternatively, you can load the C-GAP-17 dataset
# structures = load_dataset("C-GAP-17")

train, val, test = structures.random_split([0.8, 0.1, 0.1], seed=42)

In [ ]:
from ase.io import write

write("train.xyz", train)
write("valid.xyz", val)
write("test.xyz", test)

## Hyperparameters and configuration files
We provide two configuration input files (in `.yaml` format) for `graphPES` for SchNet and NequIP. Check out their structure, and finish the missing parts to train your first GNN MLIP! Feel free to experiment with different values for hyperparameters such as the `cutoff`, the number of `radial_features` and the number of `layers`. We also have two separate datasets for training and testing the models, one we used in the previous class and C-GAP-17 which you can load by uncommenting in the cells above.

Some tasks to complete:
1. Change the `cutoff` to 5.0 Angstroms. How does this affect the training time and the model performance? (_Hint: look at the error metrics in the training output_)
2. Change the number of training configurations between 10 and 500 to construct the learning curves. (_Hint: Plot the RMSE on the y axis and on x axis plot the number of structures_)
3. Change the number of `radial_features` to 64 and the number of `layers` to 2. How does this affect the training time and the model performance? (_Hint: look at the error metrics in the training output_)
4. Now train a model with the C-GAP-17 dataset and save this model separately for later. We will be comparing the performance of this model with the one trained on the previous dataset. (_Hint: use the best hyperparameters you found in the previous steps_)

In [ ]:
# train a grapPES model from the command line
# if training with the C-GAP-17 dataset, set the number of training configuration to ~1000
# set to fit on GPU 1
!graph-pes-train train_SchNet.yaml general/run_id=train-SchNet

## Evaluating the model performance
Once the model(s) has finished training, you can load it and evaluate its performance on the training and test sets. This typically involves calculating the root mean square error (RMSE) between the predicted and true energies and forces. We will be using ASE as a calculator to interface with the trained model to compute the energies and forces. For more information on how to use ASE with graph-pes, check out the documentation here:
- [ASE calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html)
- [graph-pes ASE interface](https://jla-gardner.github.io/graph-pes/tools/ase.html)

In [2]:
# load graphPES models
from graph_pes.models import load_model
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use GPU if available
# load the best model from the training
best_model = (
    load_model("graph-pes-results/train-SchNet/model.pt")  # load the model
    .to(device)  # move to GPU if available
    .eval()  # set to evaluation mode
)

# setup ASE calculator
calculator = best_model.ase_calculator()

# example of getting energies and forces using the calculator
frm = test[0]
frm.calc = calculator
energy_pred = frm.get_potential_energy()
forces_pred = frm.get_forces()

# perform the same for all structures in the test set and find the performance metric like you did in Day 1
# examine scatter plots of reference energies and forces vs ML predicted quantities
...

Ellipsis

Now you can redo the same steps to train a NequIP model (takes ~3 times longer on CPU to obtain a more accurate MLIP). To do so, use the configuration file `train_NequIP.yaml`. Save your best SchNet and NequIP models and create a `models` folder. We will be using these models in the next part of the class.

```bash
mkdir models
```


## Part 2
## Running molecular dynamics with machine learning interatomic potentials
In this section, we will be running molecular dynamics (MD) simulations using the trained MLIPs of Part 1 using the MD engine within ASE. First of all, we test the stability of the trained MLIPs by annealing (holding at constant temperature using the NVT ensemble) a few starting configurations at 300K. This is a good way to check if the model is stable and can reproduce the expected behavior of the system. We will also perform a geometric optimisation of the structures as another test of stability. We will then perform two types of MD simulations:

1. **Graphitisation**: A graphitisation simulation is a simulation where we take an initial configuration of carbon which is amorphous at a given density and ramp the temperature from 300K to a high temperature between 2000K and 4000K and anneal for hundreds of picoseconds. This is a less demanding simulation than the melt-quench (see below) but is a good test to study subtle changes in the carbon structure involving bond breaking and formation to form more ordered domains. 
2. **Melt-Quench**: A melt-quench is a simulation where we first heat the system to a high temperature (> 5000 K) and then rapidly cool it down to room temperature (e.g., 300K). This is a common technique to study the melting and crystallization of materials. This is a more demanding simulation than the graphitisation simulation, as it involves a large temperature ramp, rapid cooling and can result in catastrophic failure of the model if it is not stable.

We will be using the `ase.md` module to run the MD simulations and `ase.optimize` for the geometry optimisation. For more information on how to use ASE for MD and geometry optimisation, check out the documentation here:
- [ASE MD documentation](https://wiki.fysik.dtu.dk/ase/ase/md.html#module-ase.md)
- [ASE Optimizers documentation](https://wiki.fysik.dtu.dk/ase/ase/optimize.html)

Some tasks to complete:
1. Run a geometry optimisation using LBFGS on a random structure from the C-GAP-17 dataset using the trained SchNet model. Check the final energy and forces of the optimised structure. How do they compare to the initial values? Do the same but now using your NequIP model. Which performs better and does this make sense with the error metrics seen in the output? (_Hint: Change the convergence criterion and number of steps and see what happens_)
2. Run a NVT simulation at 300K for 10 ps on a random structure from the C-GAP-17 dataset using the trained SchNet model. Check the final energy and forces of the structure after the simulation. Do the same but now using your NequIP model. Is this structure stable? (_Hint: Visualise the trajectory using OVITO_)
3. Run a graphitisation simulation on a random structure from the C-GAP-17 dataset using the trained SchNet model. Check the final energy and forces of the structure after the simulation. Do the same but now using your NequIP model. Try different temperatures and densities and see what kind of structures emerge? (_Hint: You will want to run this for at least 100 ps_)
4. Run a melt-quench simulation on a random structure from the C-GAP-17 dataset using the trained SchNet model. Check the final energy and forces of the structure after the simulation. Do the same but now using your NequIP model. How does the structure change during the simulation? What kind of structures emerge? (_Hint: You will want to melt the structure into a liquid to thermalise it and then rapidly cool down. Find the optimal temperature. If it is unstable, try and do this using the models trained on the C-GAP-17 dataset_)

In [3]:
# load the dataset from Day 1:
starting_configs = load_dataset("../Class-1/structures_filt.xyz")

In [20]:
# load the trained MLIP

from graph_pes.models import load_model
import torch
# Set GPU environment variable to use GPU 1
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = (
    load_model("graph-pes-results/train-SchNet/model.pt")  # load the model
    .to(device)  # move to GPU if available
    .eval()  # set to evaluation mode
)
# setup ASE calculator
calculator = best_model.ase_calculator()

# test with a few indices for different densities and starting configurations
initial_frame = starting_configs[0]

# setup calculator
initial_frame.calc = calculator

In [ ]:
# Begin by performing a geometry optimization
from ase.optimize import LBFGS

# write the code to perform geometry optimization. it should be two lines of code!
# ...
opt = LBFGS(initial_frame, trajectory="opt.traj", logfile="opt.log")
opt.run(fmax=0.05, steps=100)  # run the optimizer until forces are below 0.05 eV/Ang

# write the optimized structure to a file
from ase.io import write
write("optimized_structure.xyz", initial_frame)

In [ ]:
# Try again but now with FrechetCellFilter to allow the cell to relax
from ase.filters import FrechetCellFilter

# https://wiki.fysik.dtu.dk/ase/ase/filters.html#ase.filters.FrechetCellFilter

In [14]:
# this cleans the atoms object to run

def rename_calculator_outputs(atoms):
    # Rename per-atom array keys
    if "forces" in atoms.arrays:
        atoms.arrays["GAP_forces"] = atoms.arrays["forces"]
        del atoms.arrays["forces"]

    if "stress" in atoms.arrays:
        atoms.arrays["GAP_stress"] = atoms.arrays["stress"]
        del atoms.arrays["stress"]

    # Rename global info keys
    if "energy" in atoms.info:
        atoms.info["GAP_energy"] = atoms.info["energy"]
        del atoms.info["energy"]

    if "virial" in atoms.info:
        atoms.info["GAP_virial"] = atoms.info["virial"]
        del atoms.info["virial"]
initial_frame.get_forces()  # triggers calculator
rename_calculator_outputs(initial_frame)

In [22]:
# let's run MD at 300K to thermalise our structure

import ase
from ase.md.velocitydistribution import (
    MaxwellBoltzmannDistribution,
    ZeroRotation,
    Stationary,
) 
from ase.md import MDLogger
from ase.md.nvtberendsen import NVTBerendsen
from ase.io import read

Tinit = 300  # initial temperature in Kelvin

md_params = {
    "timestep": 1 * ase.units.fs,  # MD timestep in femtoseconds
    "taut": 100 * 0.5 * ase.units.fs,  # thermostat time constant
}
total_md_steps = 10000  # make sure change this to match the appropriate time scale of your system. 

# initialize velocities
MaxwellBoltzmannDistribution(initial_frame, temperature_K=Tinit)
Stationary(initial_frame)
ZeroRotation(initial_frame)

# initialize dynamics object
dyn = NVTBerendsen(initial_frame, temperature_K=Tinit, **md_params)

# write trajectory function
def write_frame():
    dyn.atoms.write(
        f"trajectory.xyz", append=True
    )  # make sure the extension is xyz. Make sure to save the path of each trajectory file to avoid overwriting. We will analyze these later.


dyn.attach(write_frame, interval=100)  # set the frequency of writing to trajctory file

# setup the logger
dyn.attach(
    MDLogger(
        dyn,  # dynamics object
        initial_frame,  # intial configuration
        f"log.log",
        peratom=True,
        mode="a",
    ),
    interval=100,  # frequency of writing the log
)

# run the MD
dyn.run(total_md_steps)

True

Now we will perform a graphitisation by taking a structure and holding it at a high temperature for a long time. This is a good way to study the stability of the MLIPs and their ability to reproduce the expected behavior of the system. We will also perform a melt-quench simulation, which is a more demanding simulation that involves a large temperature ramp and rapid cooling.

In [ ]:
# Using the code above, try and recreate this but now for a different initial configuration and hold at high temperature for a longer time. Start with 40 ps. Save this trajectory as we will analyze it later. Try different densities and temperatures saving each trajectory with a different name.

...

In [28]:
import numpy as np
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary, ZeroRotation
from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger
from ase.io import read, write
from ase.io.trajectory import Trajectory

# === Parameters ===
Tinit = 300      # Initial temperature (K)
Tmelt = 6000     # Melting temperature (K)
Tfin = 300       # Final (quench) temperature (K)

timestep = 1.0 * units.fs  # Time step
taut = 50.0 * units.fs     # Thermostat time constant
interval = 100             # Write output every 100 steps

# Simulation steps (adjust as needed)
heating_steps = 10000
melt_hold_steps = 50000
quench_steps = 20000

# === Load initial structure ===
#initial_frame = read("trajectory.xyz", index=-1)  # or use `load_dataset()[0]` if from dataset
initial_frame.calc = calculator  # Set the calculator for the initial frame
MaxwellBoltzmannDistribution(initial_frame, temperature_K=Tinit)
Stationary(initial_frame)
ZeroRotation(initial_frame)

# === Initialize trajectory and logger ===
traj = "trajectory.xyz"
logfile = "md.log"

def write_frame():
    write(traj, dyn.atoms, append=True)

# === Create dynamics object ===
dyn = NVTBerendsen(initial_frame, temperature_K=Tinit, timestep=timestep, taut=taut)
dyn.attach(write_frame, interval=interval)
dyn.attach(MDLogger(dyn, initial_frame, logfile, mode="a", peratom=True), interval=interval)

# === Heating phase ===
print("Heating...")
heating_temps = np.linspace(Tinit, Tmelt, heating_steps // 10)
for T in heating_temps:
    dyn.set_temperature(T)
    dyn.run(10)

# === Melting / hold phase ===
print("Melting at 6000 K...")
dyn.set_temperature(Tmelt)
dyn.run(melt_hold_steps)

# === Quenching phase ===
print("Quenching...")
quench_temps = np.linspace(Tmelt, Tfin, quench_steps // 10)
for T in quench_temps:
    dyn.set_temperature(T)
    dyn.run(10)

print("Melt-quench complete.")


Heating...
Melting at 6000 K...
Quenching...
Melt-quench complete.


In [ ]:
# perform your analysis on the produced trajectory. Track sp2 and sp3 carbons, calculate the average bond length, etc using what we did in our previous class.
...